In [2]:
from kdpipelines.Pipeline import Pipeline
from kdpipelines import Adapters, Processors, Sinks
import pandas as pd
import json

/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
06/28/2021 09:45:46 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="aifb-ls3-vm1.aifb.kit.edu", username="", password="", index="document", duplicate_documents='overwrite')

06/30/2021 10:40:22 - INFO - faiss.loader -   Loading faiss with AVX2 support.
06/30/2021 10:40:22 - INFO - faiss.loader -   Loading faiss.
/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
06/30/2021 10:40:22 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
06/30/2021 10:40:23 - INFO - elasticsearch -   HEAD http://aifb-ls3-vm1.aifb.kit.edu:9200/ [status:200 request:0.058s]
06/30/2021 10:40:23 - INFO - elasticsearch -   HEAD http://aifb-ls3-vm1.aifb.kit.edu:9200/document [status:200 request:0.020s]
06/30/2021 10:40:23 - INFO - elasticsearch -   GET http://aifb-ls3-vm1.aifb.k

### Pipeline Definition

In [ ]:
adapter = Adapters.UnarxiveAdapter("/home/daniel/KIT/KD/samples/sample")
pipeline = Pipeline(adapter, batch_size=5, cpus=10)

In [ ]:
#MetadataArxiveEnricher
from arxive_metadata.rocksDB import RocksDBAdapter
db = RocksDBAdapter("arXive_metadata", "http://localhost:8089")
pipeline.add_processor(Processors.MetadataArxiveEnricher("arixive-id",db))

In [ ]:
#FilterOnMetadataValue
cs_values = ["cs.AI", "cs.AR", "cs.CC", "cs.CE", "cs.CG", "cs.CL", "cs.CR","cs.CV","cs.CY","cs.DB","cs.DC","cs.DL","cs.DM","cs.DS","cs.ET","cs.FL","cs.GL","cs.GR", "cs.GT","cs.HC","cs.IR","cs.LG","cs.LO","cs.MA","cs.MM","cs.MS","cs.NA","cs.NE","cs.NI","cs.OH","cs.OS","cs.PF","cs.PL","cs.RO","cs.SC","cs.SD","cs.SE","cs.SI","cs.SY"]
pipeline.add_processor(Processors.FilterOnMetadataValue("categories", cs_values))

In [ ]:
#Clean up text
#TextKeywordCut Introduction
pipeline.add_processor(Processors.TextKeywordCut("Introduction"))
#TextKeywordCut Acknowledgements
pipeline.add_processor(Processors.TextKeywordCut("Acknowledgments", False))
#TextAppendMetadataField
pipeline.add_processor(Processors.TextAppendMetadataField("abstract", True))

In [ ]:
#MetadataFieldDiscarder
fields_to_discard = ["update_date", "authors", "abstract", "versions", "comments", "report-no", "license", "id", "categories", "submitter", "journal-ref", "authors_parsed"]
pipeline.add_processor(Processors.MetadataFieldDiscarder(fields_to_discard))

In [ ]:
#MetadataMagArxiveLinker
import pandas as pd
df = pd.read_csv("../../data/unarxive_metadata/mag_id_2_arxiv_id.csv", names=["mag_id", "idk", "url", "arxive_id"])
pipeline.add_processor(Processors.MetadataMagArxiveLinker(df, column_to_match="arxive_id", column_to_add="mag_id", field_to_match="arixive-id"))
del df

In [ ]:
#TextReplaceFilter
pipeline.add_processor(Processors.TextReplaceFilter(r"\{{(.*?)\}}", ""))

In [ ]:
#HaystackPreProcessor
from haystack.preprocessor import PreProcessor
pre_processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=0)
pipeline.add_processor(Processors.HaystackPreProcessor(pre_processor))

In [ ]:
#TextfileSink
#pipeline.add_sink(Sinks.TextfileSink("/home/daniel/KIT/KD/testoutput", ["arixive-id", "_split_id"]))
pipeline.add_sink(Sinks.TextfileSink("/home/daniel/KIT/KD/testoutput", ["arixive-id"]))

In [ ]:
#ElasticsearchSink
pipeline.add_sink(Sinks.ElasticsearchSink(document_store))

In [ ]:
#PrintSink
pipeline.add_sink(Sinks.PrintSink())

In [ ]:
print(pipeline)

### Use Pipeline to process documents

In [ ]:
pipeline.process()

### Pipeline to update entries in Elasticsearch

In [ ]:
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10)

In [ ]:
update_pipeline = Pipeline(e_adapt, batch_size=20, cpus=10)

In [4]:
#ElasticsearchSink (put items back into Elasticsearch)
update_document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")
update_pipeline.add_sink(Sinks.ElasticsearchSink(update_document_store))

06/30/2021 10:40:33 - WARNING - elasticsearch -   HEAD http://localhost:9200/ [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/elasticsearch/connection/http_urllib3.py", line 246, in perform_request
    method, url, body, retries=Retry(False), headers=request_headers, **kw
  File "/home/daniel/anaco

06/30/2021 10:40:33 - WARNING - elasticsearch -   HEAD http://localhost:9200/ [status:N/A request:0.000s]
Traceback (most recent call last):
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/urllib3/connection.py", line 157, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/urllib3/util/connection.py", line 84, in create_connection
    raise err
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/urllib3/util/connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/daniel/anaconda3/envs/KD/lib/python3.7/site-packages/elasticsearch/connection/http_urllib3.py", line 246, in perform_request
    method, url, body, retries=Retry(False), headers=request_headers, **kw
  File "/home/daniel/anaco

ConnectionError: Initial connection to Elasticsearch failed. Make sure you run an Elasticsearch instance at `[{'host': 'localhost', 'port': 9200}]` and that it has finished the initial ramp up (can take > 30s).

In [ ]:
print(update_pipeline)

In [ ]:
update_pipeline.process()

In [ ]:
len(adapter)

In [ ]:
len(e_adapt)

In [ ]:
#IMRaD classification
from imrad_classification import BERTClassificationHandler
classification_handler = BERTClassificationHandler("/home/daniel/BERT_copy.model")
pipeline.add_processor(Processors.IMRaDClassification(classification_handler))

In [ ]:
adapter = Adapters.UnarxiveAdapter("/home/daniel/KIT/KD/sample")

In [ ]:
docs = adapter.generate_documents(50)

In [ ]:
type(docs[0])

In [ ]:
datasets = pd.read_json("../../data/pwc_metadata/datasets_preprocessed.json")
datasets = {}
for index, row in datasets.iterrows():
    datasets[row['itemLabel']] = row['item']
pipeline.add_processor(Processors.StringMatchingProcessor(datasets, 'methods', 'wikidata_id'))

In [ ]:
adapter = Adapters.GrobidAdapter("/home/daniel/KIT/KD/unpaywall-grobid-sample")

In [ ]:
adapter.generate_documents(5)

#### Grobid Testpipeline

In [ ]:
grobid_adapter = Adapters.GrobidAdapter("/home/daniel/KIT/KD/unpaywall-grobid-sample")
grobid_pipeline = Pipeline(grobid_adapter, batch_size=2, cpus=10)

In [ ]:
#HaystackPreProcessor
from haystack.preprocessor import PreProcessor
pre_processor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
    split_overlap=0)
grobid_pipeline.add_processor(Processors.HaystackPreProcessor(pre_processor))

In [ ]:
with open('../../data/pwc_metadata/datasets_preprocessed.json') as f:
    datasets = json.load(f)
grobid_pipeline.add_processor(Processors.StringMatchingProcessor(datasets, 'datasets', 'pwc_url'))

In [ ]:
with open('../../data/pwc_metadata/methods_preprocessed.json') as f:
    methods = json.load(f)
grobid_pipeline.add_processor(Processors.StringMatchingProcessor(methods, 'methods', 'pwc_url'))

In [ ]:
grobid_pipeline.add_sink(Sinks.TextfileSink("/home/daniel/KIT/KD/testoutput", ["mag_id", "_split_id"]))

In [ ]:
print(grobid_pipeline)

In [ ]:
grobid_pipeline.process()

In [ ]:
len(grobid_adapter)

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
words = word_tokenize("Specificity Sensitivity MSA 63% 90% RBF-single 66% 2.7% 84% ±2.0% MLP-single 61% ±0.0% 91% ±0.2%")

In [ ]:
num = []
for word in words:
    if word.isnumeric() or word.isalnum():
        num.append(word)

In [ ]:
import re
re.findall("(\d+)", "Specificity Sensitivity MSA 63% 90% RBF-single 66% 2.7% 84% ±2.0% MLP-single 61% ±0.0% 91% ±0.2%")

In [ ]:
"Hello\nthis\nis\na test".split("\n")

In [ ]:
grobid_pipeline.add_processor(Processors.SplitByLinePreProcessor())

In [ ]:
print(grobid_pipeline)

In [ ]:
p = Processors.SplitByLinePreProcessor()

In [ ]:
grobid_adapter.reset()
docs = grobid_adapter.generate_documents(10)

In [ ]:
docs_split =  Processors.SplitByLinePreProcessor().process(docs)

In [ ]:
docs_split

In [ ]:
docs_discarded = Processors.DiscardLineProcessor(0.5).process(docs)

In [ ]:
import enchant

In [ ]:
words = word_tokenize(docs[0]['text'])

In [ ]:
words

In [ ]:
num_non = 0
for word in words:
    if not word.isalpha():
        num_non += 1

In [ ]:
docs

In [ ]:
docs_discarded

In [ ]:
"a".isalpha()

In [ ]:
len(e_adapt)

In [ ]:
e_adapt.generate_documents(10)

In [ ]:
filters = {'arixive-id':["1805.03106"]}

In [ ]:
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10, filters=filters)

In [ ]:
test_pipeline = Pipeline(e_adapt, batch_size=2, cpus=10, max_runtime=1)

In [ ]:
test_pipeline.add_processor(Processors.MetadataFieldAdder("_status", 0))

In [ ]:
test_pipeline.add_sink(Sinks.ElasticsearchSink(document_store))

In [ ]:
print(test_pipeline)

In [ ]:
test_pipeline.process()

In [ ]:
e_adapt.reset()

In [ ]:
e_adapt.generate_documents(1)

In [ ]:
import re
any_filter = {'status':["0"]}

In [ ]:
test_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10, filters=any_filter)

In [ ]:
test_adapt.generate_documents(1)

In [ ]:
import re
def _string_pattern_percentage(s:str, filter):
    words = len(word_tokenize(s))
    matches = len(re.findall(filter, s))
    return matches/(words*1.0)

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
_string_pattern_percentage("Die FORMULA ist eine FORMULA mit FORMULA.", "FORMULA")

In [ ]:
with open("/home/daniel/KIT/KD/samples/sample/0803.3704.txt", "r") as f:
    lines = f.readlines()

In [ ]:
text = "".join(lines)
paragraphs = text.split("\n\n")

In [ ]:
p_temp= []
for paragraph in paragraphs:
    lines = paragraph.split("\n")
    if lines:
        if len(word_tokenize(lines[0])) < 5:
            lines.remove(lines[0])
        elif not lines[0].endswith("."):
            lines.insert(0, lines[0] + ".")
            lines.pop(1)
    p_temp.append(" ".join(lines))

In [ ]:
p_temp

In [ ]:
"Die FORMULA ist eine FORMULA mit FORMULA.".replace("FORMULA", "")

In [ ]:
docs = adapter.generate_documents(5)

In [ ]:
docs[4]['text'].split("\n\n")

In [ ]:
"\ntest\ntest\n\n".strip("\n")

Update and add _status

In [ ]:
e_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10000)
update_pipeline = Pipeline(e_adapt, batch_size=10000, cpus=5)

In [ ]:
update_pipeline.add_processor(Processors.MetadataFieldAdder("_status", 0))

In [ ]:
update_pipeline.add_sink(Sinks.ElasticsearchSink(document_store))

In [ ]:
update_pipeline.add_sink(Sinks.TextfileSink("/home/daniel/KIT/KD/testoutput", ["mag_id", "_split_id"]))

In [ ]:
update_pipeline.process()

In [ ]:
print(update_pipeline)

In [ ]:
update_pipeline._adapter.reset()

In [ ]:
e_adapt.reset()

In [ ]:
len(e_adapt)

In [ ]:
new_adapt = Adapters.ElasticsearchAdapter(document_store, batch_size=10000, filters={'_status':["0"]})

In [ ]:
len(new_adapt)